In [30]:
%pylab
from importlib import reload
import particleFilter
reload(particleFilter)
from particleFilter import forwardSim, MyParticle, ParticlePopulation, multivariate_uniform,get_weight_single_particle


Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


# creat some toydata

In [31]:
traj =[]
dt = 10
rates = [0.5, 0.05, 0.5, 0.05]
traj.append([0,0])

s,_,_ = forwardSim(iniT=0, iniS=traj[-1], maxTime=dt, rates=rates)
traj.append(s)

In [32]:
# build some observed trajectory
for i in range(10):
    s, _,_ = forwardSim(iniT=i*dt, iniS=traj[-1], maxTime=i*dt+dt, rates=rates)
    traj.append(s)

# build the prior population

In [33]:
lB, uB = np.array([-2,-4,-2,-4]), np.array([0,0,0,0]) # lower and upper bounds of parameters: natural log!!

nParticles = 1000
prior_thetas = multivariate_uniform(nParticles, lB,uB) #the sampling

prior = ParticlePopulation()
for i in range(nParticles):
    p = MyParticle(prior_thetas[i,:], [0,0])
    prior.addParticle(p, 1)

# the paricle filtering

In [36]:
nParticles = 1000
populations = [prior]

prior_alpha = 1
prior_beta = 1

for i in range(1, len(traj)): # for one time step: oen pop
    print('Population ', i)
    newPopulation = ParticlePopulation()
    
    for n in range(nParticles):
        
        #sampel a particle from the previous population
        newPart = next(populations[-1].sample_from_population_generator(1)) # returns an generator
        
        p,state,counter,G = newPart.get_weight_single_particle(tau=dt, datapoint =traj[i][1] )
        
        #print(p,state, newPart.theta)
        #print('////')
        
        #update the particle (ie updating the gamma-parameters and the state)
#         newTheta = np.random.gamma(shape=counter+prior_alpha,scale=1/(G+prior_beta))
#         newAlpha = newPart.theta[0:4] + counter 
#         newBeta = newPart.theta[5:] + G
        
#         pa = MyParticle(np.concatenate((newAlpha, newBeta),0), state)
        pa = MyParticle(newPart.theta, state)
        newPopulation.addParticle(pa, p)
        
    populations.append(newPopulation)

Population  1
Population  2
Population  3
Population  4
Population  5
Population  6
Population  7
Population  8
Population  9
Population  10
Population  11


In [37]:
populations[-1].plot_population_sample(1000)

In [38]:
populations[3].plot_population_sample(1000)

In [39]:
# plot the hidden state trajectory
nSamples = 1000
RNA_over_time = zeros((nSamples, len(traj)))
for i, pop in enumerate(populations):
    m = pop.sample_from_population_matrix(nSamples)
    RNA_over_time[:,i] = m[:,4]
import seaborn as sns
sns.violinplot(data=RNA_over_time)
plot(traj)        

In [41]:
# plot the parameters trajectory
nSamples = 500
theta_over_time = zeros((nSamples, len(traj)))
for i, pop in enumerate(populations):
    m = pop.sample_from_population_matrix(nSamples)
    theta_over_time[:,i] = m[:,1]
import seaborn as sns
figure();
sns.violinplot(data=theta_over_time, inner="stick")

/home/michi/anaconda3/envs/NLP/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
